КОРОЧЕ
В этом ноутбуке я пытался обучить ast - трансформер для аудио классификации, но он жрет овер 14гб в текущей конфигурации

In [1]:
DATA_PATH = '../data'
TRAIN_DATA_PATH = f'{DATA_PATH}/train'
TASK_DATA_PATH = f'{DATA_PATH}/test'
TRAIN_FILENAME = f'{DATA_PATH}/train_gt.csv'
TASK_FILENAME = f'{DATA_PATH}/test.csv'

In [2]:
import pandas as pd
from transformers import AutoFeatureExtractor, ASTForAudioClassification, Trainer, ASTConfig, ASTModel, TrainingArguments, ASTFeatureExtractor
from datasets import Dataset, load_dataset, Audio

/opt/miniconda3/envs/tune-ast/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv(TRAIN_FILENAME, header=None, names=['audio', 'label'])
df['audio'] = TRAIN_DATA_PATH + "/" + df['audio']

dataset = Dataset.from_pandas(df).cast_column("audio", Audio())
sampling_rate = dataset.features["audio"].sampling_rate

[ASTConfig](https://huggingface.co/docs/transformers/model_doc/audio-spectrogram-transformer#transformers.ASTConfig)
Думаю можно подобрать параметра полегче

In [4]:
model_config = ASTConfig(
    hidden_size = 256,
    num_hidden_layers = 4, # 12
    num_attention_heads = 4, # 12
    intermediate_size = 1024,
    hidden_act = 'gelu',
    hidden_dropout_prob = 0.0,
    attention_probs_dropout_prob = 0.0,
    initializer_range = 0.02,
    layer_norm_eps = 1e-12,
    patch_size = 16,
    qkv_bias = True,
    frequency_stride = 10,
    time_stride = 10,
    num_mel_bins = 128
)

model = ASTForAudioClassification(model_config)
#model = ASTForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

feature_extractor = ASTFeatureExtractor()

/opt/miniconda3/envs/tune-ast/lib/python3.11/site-packages/transformers/audio_utils.py:297: UserWarning: At least one mel filter has all zero values. The value for `num_mel_filters` (128) may be set too high. Or, the value for `num_frequency_bins` (256) may be set too low.
  warnings.warn(


In [5]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Preprocess the dataset
def preprocess_function(examples):
    audio = examples["audio"]  # Adjust based on your dataset structure
    inputs = feature_extractor(audio["array"], return_tensors='np', feature_size=-1, sampling_rate=16000) # Probably not 16000
    inputs['input_values'] = inputs['input_values'].reshape(1024, 128)
    return inputs

# Metrics computing while fine tuning
def compute_metrics(p):
    predictions, labels = p

    argmax_predictions = np.argmax(predictions, axis=1)

    precision = precision_score(labels, argmax_predictions, zero_division=0)
    recall = recall_score(labels, argmax_predictions, zero_division=0)
    f1 = f1_score(labels, argmax_predictions, zero_division=0)
    acc = accuracy_score(labels, argmax_predictions)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "accuracy": acc,
    }

In [6]:
encoded_dataset = dataset.map(preprocess_function, remove_columns=["audio"]).train_test_split(test_size=0.3)

Map: 100%|██████████| 8803/8803 [01:41<00:00, 86.66 examples/s] 


In [9]:
training_arguments = TrainingArguments(
    output_dir="ast_save",
    learning_rate=0.01,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=0,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/miniconda3/envs/tune-ast/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
0it [00:00, ?it/s]

{'train_runtime': 0.0017, 'train_samples_per_second': 0.0, 'train_steps_per_second': 0.0, 'train_loss': 0.0, 'epoch': 0}


TrainOutput(global_step=0, training_loss=0.0, metrics={'train_runtime': 0.0017, 'train_samples_per_second': 0.0, 'train_steps_per_second': 0.0, 'total_flos': 0, 'train_loss': 0.0, 'epoch': 0})

In [7]:
task_df = pd.read_csv(TASK_FILENAME, header=None, names=['audio', 'label'])
task_df['audio'] = TASK_DATA_PATH + "/" + task_df['audio']

task_dataset = Dataset.from_pandas(task_df).cast_column("audio", Audio())
sampling_rate = task_dataset.features["audio"].sampling_rate

In [8]:
task_encoded_dataset = task_dataset.map(preprocess_function, remove_columns=["audio"])

Map: 100%|██████████| 2870/2870 [00:35<00:00, 80.97 examples/s] 


In [10]:
pred = trainer.predict(task_encoded_dataset)
pred

100%|██████████| 180/180 [03:17<00:00,  1.10s/it]


PredictionOutput(predictions=array([[0.10181832, 0.4093188 ],
       [0.16138676, 0.36548704],
       [0.10461596, 0.38973323],
       ...,
       [0.00888395, 0.42633086],
       [0.1751698 , 0.36425513],
       [0.18473291, 0.38646677]], dtype=float32), label_ids=None, metrics={'test_runtime': 199.4017, 'test_samples_per_second': 14.393, 'test_steps_per_second': 0.903})

In [11]:
pred_argmax = np.argmax(pred[0], axis=1)

In [12]:
np.sum(pred_argmax)

0

In [13]:
assert len(pred_argmax) == len(task_df)